<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/exposure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from lxml import html
import pandas as pd
import json
import numpy as np
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fileid = '1JokoV68YD5Iq2l4Y_IV2RJzBpD_mSCyq'
download = drive.CreateFile({'id': fileid})
download.GetContentFile('user.json')

In [0]:
ranking = get_ranking_from_call("https://www.yelp.com/biz/ci0-GO_iMb0NdTCUYWjocQ","en", "relevance_desc", "")
# ranking = get_ranking("https://www.yelp.com/biz/noche-de-margaritas-new-york")
# users = [x[1] for x in ranking]  # set of user
print(len(ranking))
# df_ranking = find_users_in_dataset(ranking)  # cannot find all users
df_ranking = create_df_users(ranking)

print("\n++++++++++++++++ RANKING ++++++++++++++++++\n")

print("Ranking by Yelp filter:")
print(df_ranking)

print("\n++++++++++++++++ GROUPS CREATION ++++++++++++++++++\n")
attribute = 'Review_Count'
print("ATTRIBUTE: " + attribute + "\n")
group_list = create_objective_groups_by_size(df_ranking, attribute, 2)
index = 1
for group in group_list:
    print("\nGROUP " + str(index) + ":")
    for person in group:
        print(person['user_id'] + ", " + person['Review_count'])
    index = index + 1

print("\n++++++++++++++++ EXPOSURE CALCULATION ++++++++++++++++++\n")
index = 1
for group in group_list:
    print("\nEXPOSURE of group " + str(index) + " is: " + str(exposure(group, df_ranking)))
    index = index + 1



[]
N. of reviews = 0
[]
N. of reviews = 0
[]
N. of reviews = 0
0

++++++++++++++++ RANKING ++++++++++++++++++

Ranking by Yelp filter:
[]

++++++++++++++++ GROUPS CREATION ++++++++++++++++++

ATTRIBUTE: review_count

Number of groups created: 29

GROUP 1:
Arielle, IrcuN4TKWkPBPyDaQ_VTHw
Kathleen, u4IpL4LnF4s-LLA_DD316g

GROUP 2:
Bea, Zzki9YMObPa19AmjetqMOg
Amanda, 1ui4NwkXZk5Y9aYPKTFdRA

GROUP 3:
Alex, NLnxvTdm5MC8JBzlRZYtug
Kate, d1oQPhZKiU5bxWQYkGxiwQ

GROUP 4:
Shawn, P-gLhB0mfZF4xYr-SE4UZA
Stephen, JLX6s_BzWu6DK7vzuS35sg

GROUP 5:
Zain, rukix1QxEpNH2Epj2FG0IA
Irish, aVRPfNY7aaR3DZtAkQY_Hw
Justin, om59YhqVsAvluMSwVj6mYw

GROUP 6:
Eaton, oIQPQtOux-XaZIj9Q2_uTA
Annette, ABgQuo2VN-46P3mV7g0-KA
Latika, MIDUK97UMuGKOido3cxjHg

GROUP 7:
Susan, DL-KW73oKYwmuQV2LW-unQ
Mel, tsnH60AmDpkIR_Rv3VZaMw

GROUP 8:
Danielle, Fi8O--oVgpLgXUOIQlgs4w
Tanya, 4E6WGB8XZrSa_26UcVTSvQ

GROUP 9:
Niccole, mBdiqpmjfD1TH3UDKGuQNw
Heather, 8KOmmZGCVC9XwhyRR3cUPg

GROUP 10:
Tatiana, DbQnWxwY6or04Ig4KcmWyQ
Stacey, U

In [0]:
def get_ranking_from_call(url_business, lang, sort, query):
  
  headers = [{"name":"Accept","value":"*/*"},{"name":"Accept-Encoding","value":"gzip, deflate, br"},{"name":"Accept-Language","value":"it-IT,it;q=0.8,en-US;q=0.5,en;q=0.3"},{"name":"Connection","value":"keep-alive"},{"name":"Content-Type","value":"application/x-www-form-urlencoded; charset=utf-8"},{"name":"Cookie","value":"qntcst=D; hl=en_US; wdi=1|3C26116D69138F61|0x1.78d019f71a444p+30|a7756ff94751d3a9; _ga=GA1.2.3C26116D69138F61; location=%7B%22city%22%3A+%22New+York%22%2C+%22state%22%3A+%22NY%22%2C+%22country%22%3A+%22US%22%2C+%22latitude%22%3A+40.713%2C+%22longitude%22%3A+-74.0072%2C+%22max_latitude%22%3A+40.8523%2C+%22min_latitude%22%3A+40.5597%2C+%22max_longitude%22%3A+-73.7938%2C+%22min_longitude%22%3A+-74.1948%2C+%22zip%22%3A+%22%22%2C+%22address1%22%3A+%22%22%2C+%22address2%22%3A+%22%22%2C+%22address3%22%3A+null%2C+%22neighborhood%22%3A+null%2C+%22borough%22%3A+null%2C+%22provenance%22%3A+%22YELP_GEOCODING_ENGINE%22%2C+%22display%22%3A+%22New+York%2C+NY%22%2C+%22unformatted%22%3A+%22New+York%2C+NY%2C+US%22%2C+%22accuracy%22%3A+4.0%2C+%22language%22%3A+null%7D; xcj=1|Ptt9P03gfc75x_PBT9zmqCkUuSuyB7PR-wWUBvABNi4; __qca=P0-60561249-1581956668708; G_ENABLED_IDPS=google; __cfduid=db8764ff59d8028a6c2e1b214867927d81583160194; _gid=GA1.2.2014867238.1583835527; bse=05dcd9d5de304ef0b1d9a76fa768b10f; sc=8a1ca0dbc2; pid=505721aa4569e7bb"},{"name":"Host","value":"www.yelp.com"},{"name":"Referer","value":"https://www.yelp.com/biz/noche-de-margaritas-new-york"},{"name":"TE","value":"Trailers"},{"name":"User-Agent","value":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0"},{"name":"X-Requested-By-React","value":"true"},{"name":"X-Requested-With","value":"XMLHttpRequest"}]
  headers_ok = {}
  for header in headers:
    temp = {
        header['name']: header['value']
    }
    headers_ok.update(temp)
  
  x = 0
  reviews_list = []
  position = 1
  url = url_business + "/review_feed?rl=" + lang + "&sort_by=" + sort + "&q=" + query
  
  while 1:
    if x == 0:
      page_load = requests.get(url + '&start=', headers=headers_ok)
    else:
      page_load = requests.get(url + '&start=' + str(x), headers=headers_ok)
    print(page_load)
    x = x + 20
    reviews = page_load.json()['reviews']
    print(json.dumps(reviews, indent=4, sort_keys=True))
    if reviews == []:
      break
    for review in reviews:
      reviews_list.append((position, review['userId']), review['user']['review_count'])
      position = position + 1
  print(reviews_list)
  return reviews_list


In [0]:
def create_df_users(ranking):
  df_ranking = pd.DataFrame(ranking, columns=['Position', 'user_id', 'Review_Count'])
  return df_ranking

In [0]:
def create_objective_groups_by_size(df_users, attribute, range):
    # order df_users by review_count
    df_users = df_users.sort_values(attribute).reset_index(drop=True)
    group_list = [[]]
    i = 0
    group_index = 0
    prec = -1
    for index, user in df_users.iterrows():
        if i == range:
            i = 0
            group_list.append([])
            group_index = group_index + 1
        temp = user[attribute]
        if temp != prec:
            prec = temp
            i = i + 1
        group_list[group_index].append(user)
    print("Number of groups created: " + str(group_index+1))
    return group_list


In [0]:
def exposure(group, ranking):
    return sommatory(group, ranking)/(len(group))


def sommatory(group, ranking):  # ranking is a list of user_id
    sum = 0
    for user in group:
        user_id = user['user_id']
        position = ranking.loc[ranking['user_id'] == user_id, 'Position'].tolist()[0]
        # position = ranking.index[ranking['user_id'] == user_id].tolist()[0]
        # position = position + 1
        sum = sum + exp(position)
    return sum


def exp(position):
    if position == 'no match':
        return 0
    else:
        return 1/np.log(1 + position)


def demographic_parity(g1, g2, s):
    ex1 = exposure(g1, s)
    ex2 = exposure(g2, s)
    print("|" + str(ex1) + " - " + str(ex2) + "| < 0.1?")
    return np.absolute(ex1 - ex2) < 0.1  # in our case, g1 and g2 are groups of author (reviewer)


def app(author, df):
    return df[df['UserId'] == author].iloc[0, -1]


In [0]:
# TODO: get review_count from web page
def get_ranking(url):

  i=1
  reviews_text = []
  reviews_date = []
  reviews_userid = []
  x=0

  while 1:
    if x == 0:
      page_content = requests.get(url)
    else:
        page_content = requests.get(url + '?start=' + str(x))
    x = x + 20
    
    tree = html.fromstring(page_content.content)
    recommended_reviews_text = "Recommended Reviews"
    reviews_list = tree.xpath('//section[div[div[h3[text()="%s"]]]]/div/div/ul/*' % recommended_reviews_text)
    if reviews_list == []:
      break
    else:
      # Index for ranking
      j=1
      for review in reviews_list:
          reviews_text.append((i, tree.xpath('//section[div[div[h3[text()="%s"]]]]/div/div/ul/li[%d]/div/div[last()]/div[p]/p/span' % (recommended_reviews_text, j))[0].text))
          reviews_date.append((i, tree.xpath('//section[div[div[h3[text()="%s"]]]]/div/div/ul/li[%d]/div/div[last()]/div[1]/div/div[2]/span' % (recommended_reviews_text, j))[0].text))
          user_link = tree.xpath('//section[div[div[h3[text()="%s"]]]]/div/div/ul/li[%d]/div/div/div/div/div/div/div/a/@href' % (recommended_reviews_text, j))[0]
          reviews_userid.append((i,user_link[user_link.find('=')+1:]))
          i = i + 1
          j = j + 1


  print(reviews_text)
  print("N. of reviews = " + str(len(reviews_text)))
  print(reviews_date)
  print("N. of reviews = " + str(len(reviews_date)))
  print(reviews_userid)
  print("N. of reviews = " + str(len(reviews_userid)))
  return reviews_userid


In [0]:
def find_users_in_dataset(users):
    data = []
    with open("user.json", 'r') as file:
        for line in file:
            json_data = json.loads(line)
            data.append(json_data)
    df = pd.DataFrame(data)
    df_ranking = pd.DataFrame(users, columns=['Position', 'user_id'])
    df_merged = df_ranking.merge(df, on='user_id')
    # temp = df[df['user_id'].isin(users)]
    return df_merged


def exists_in_dataset(userid):
  found = False
  with open("user.json", 'r') as file:
        for line in file:
            json_data = json.loads(line)
            if json_data['user_id'] == userid:
              found = True
  if found:
    print("Trovato")
  else:
    print("Non trovato")
  return found


def find_users_in_chopped_dataset(users):
    found = False
    data = []
    i = 1
    j = 100000
    df_users = pd.DataFrame()
    while not found:
        path = "user" + str(i) + "-" + str(j) + ".json"
        try:
            with open(path, 'r') as file:
                for line in file:
                    json_data = json.loads(line)
                    data.append(json_data)
            df = pd.DataFrame(data)
        except FileNotFoundError:
            print("Cannot find all users into the dataset")
            return 0
        else:
            temp = df[df['user_id'].isin(users)]  # df of users of the ranking
            df_users = df_users.append(temp)
            if df_users.shape[0] == len(users):
                found = True
            else:
                i = i + 100000
                j = j + 100000
    return df_users
